In [18]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-together


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv("TOEGETHER_API_KEY")

from llama_index.embeddings.together import TogetherEmbedding

embed_model = TogetherEmbedding(
    model_name="togethercomputer/m2-bert-80M-8k-retrieval", api_key=api_key
)

In [34]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
from llama_index.core import Settings
import chromadb

In [47]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")

In [14]:

from pathlib import Path
from llama_index.readers.file import PyMuPDFReader


loader = PyMuPDFReader()
documents = loader.load(file_path="./data/adhd.pdf")

print(len(documents))
print(documents[0])

8
Doc ID: 2058d919-99e7-4bd0-b10d-4b11a73775d6
Text: See discussions, stats, and author profiles for this publication
at: https://www.researchgate.net/publication/317604534 The impact of
attention deﬁcit hyperactivity disorder (ADHD) in adulthood: a
qualitative study Article  in  Irish Journal of Psychological Medicine
· June 2017 DOI: 10.1017/ipm.2017.21 CITATIONS 18 READS 40,738 4
authors, inclu...


In [59]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [46]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(
  separator=" ",
  chunk_size=1024,
  chunk_overlap=20,
  paragraph_separator="\n\n\n",
  secondary_chunking_regex="[^,.;。]+[,.;。]?",

)


In [19]:
from llama_index.llms.together import TogetherLLM


/home/abenittz/workspace/icog-training/rag-app/Rag-training2/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [20]:
llm = TogetherLLM(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=api_key
)

In [48]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

In [52]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("What is adhd?")
print(response)


 Based on the information provided, ADHD refers to Attention Deficit Hyperactivity Disorder. It is a condition that affects a person's ability to concentrate, control impulsive behavior, and in some cases, can also lead to hyperactivity. Participants in the study mentioned how their symptoms have impacted their lives, including their education, career, and self-esteem. Some participants also noted the benefits of their ADHD symptoms, such as being impulsive and easily distracted, which they found contributed to their unique personality and exciting experiences.


In [54]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [63]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = node_parser


In [60]:
text = "hello. how are you? I am fine!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [62]:
print([x.text for x in nodes])
print(nodes[1].metadata["window"])

['hello. ', 'how are you? ', 'I am fine!  ']
hello.  how are you?  I am fine!  


In [ ]:
sentence_index = VectorStoreIndex.from_documents(
    document, embed_model=embed_model
)